https://wiki.openstreetmap.org/wiki/Key:amenity?uselang=en

bar, biergarten, cafe, fast_food, food_court, ice_cream, pub, restaurant

Personally, I'm only interested in cafes. 

Downloaded 

drenthe-latest.osm.pbf   
gelderland-latest.osm.pbf
groningen-latest.osm.pbf 
limburg-latest.osm.pbf   
overijssel-latest.osm.pbf

ChatGPT suggested:

osmium tags-filter -v -o cafes.osm.pbf drenthe-latest.osm.pbf gelderland-latest.osm.pbf groningen-latest.osm.pbf limburg-latest.osm.pbf overijssel-latest.osm.pbf n/amenity=cafe

Which didn't work. Then it suggested:

osmium merge drenthe-latest.osm.pbf gelderland-latest.osm.pbf groningen-latest.osm.pbf limburg-latest.osm.pbf overijssel-latest.osm.pbf -o combined.osm.pbf

osmium tags-filter -o cafes.osm.pbf combined.osm.pbf n/amenity=cafe


In [1]:
import osmium
import folium
from folium.plugins import MarkerCluster

# Create a handler to parse the cafes from the pbf file
class CafeHandler(osmium.SimpleHandler):
    def __init__(self):
        super().__init__()
        self.cafes = []

    def node(self, n):
        # Check if the node has the 'amenity' tag and it's a cafe
        if 'amenity' in n.tags and n.tags['amenity'] == 'cafe':
            self.cafes.append((n.location.lat, n.location.lon))

# Initialize the handler and apply it to the cafes.osm.pbf file
handler = CafeHandler()
handler.apply_file("cafes.osm.pbf")

cafes = handler.cafes

# Now `handler.cafes` contains a list of (latitude, longitude) tuples for cafes
print(f"Total cafes found: {len(cafes)}")

Total cafes found: 1967


In [2]:
# Define the initial location for the map (center of the region)
min_lat = min([loc[0] for loc in cafes])
max_lat = max([loc[0] for loc in cafes])
min_lon = min([loc[1] for loc in cafes])
max_lon = max([loc[1] for loc in cafes])

initial_location = ((min_lat+max_lat)/2, (min_lon+max_lon)/2)

# Create a Folium map centered on the initial location
map_cafes = folium.Map(location=initial_location, zoom_start=7)

# marker_cluster = MarkerCluster().add_to(map_cafes)
# # Add each cafe location as a marker on the map
# for lat, lon in cafes:
#     icon = folium.Icon(prefix='fa', icon='mug-saucer', color='green')
#     folium.Marker(
#         location=(lat, lon),
#         icon=icon
#     ).add_to(marker_cluster)
for lat, lon in cafes:
    folium.CircleMarker(
        location=(lat, lon),
        radius=1,
        color="limegreen"
    ).add_to(map_cafes)

map_cafes

This takes forever

In [3]:
import igraph as ig
from pyproj import Geod, CRS

geod = CRS("EPSG:32632").get_geod()

def calculate_distance(cafe1, cafe2):
    lons = [cafe1[1], cafe2[1]]
    lats = [cafe1[0], cafe2[0]]
    return geod.line_length(lons, lats)


pieterburen = (53.3987670, 6.4549632)
pietersberg = (50.8268747, 5.6874837)

cafes.append(pieterburen)
cafes.append(pietersberg)

g = ig.Graph()
g.add_vertices(len(cafes))

distances = []
for i in range(len(cafes)):
    for j in range(i + 1, len(cafes)):  # Avoid duplicate pairs
        # Calculate the distance between cafes[i] and cafes[j]
        distance = calculate_distance(cafes[i], cafes[j]) / 1_000
        if distance <= 30:  # Add an edge if distance is less than or equal to 30 km
            g.add_edge(i, j)
            distances.append(distance)

# Add edges to the graph
g.add_edges(edges)
g.es["length"] = distances  # Add distances as the 'length' attribute for each edge

print(f"Total nodes: {g.vcount()}")
print(f"Total edges (within 30 km): {g.ecount()}")

KeyboardInterrupt: 

In [ ]:
g.get_shortest_paths(len(cafes) - 1, len(cafes), output='epath')